In [8]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        select f.name, 
        case when b.memid = 0 then sum(f.guestcost * b.slots)
        else sum(f.membercost * b.slots) end as fac_rev
        from facilities as f
        left join bookings as b
        on f.facid = b.facid
        group by f.facid
        having fac_rev <1000
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks


In [29]:
from sqlalchemy import create_engine
eng = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con = eng.connect()
query1 = """
with r as (
select f.name, 
case 
	when b.memid = 0 then f.guestcost * b.slots
    else f.membercost * b.slots end as fac_rev
from facilities as f
left join bookings as b
on f.facid = b.facid
)
select name, sum(fac_rev) as tot_rev
from r
group by name
having sum(fac_rev) < 1000
order by tot_rev
        """
rs = con.execute(query1)
print(rs.fetchall())

[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]


In [31]:
query1 = """
select r.firstname||" "||r.surname as Recommndee, l.firstname||" "||l.surname as Recomender
from members as l
left join members as r
on l.memid = r.recommendedby
where r.recommendedby is not null and l.memid <> 0
order by r.surname||r.firstname
"""
rs = con.execute(query1)
print(rs.fetchall())

[('Florence Bader', 'Ponder Stibbons'), ('Anne Baker', 'Ponder Stibbons'), ('Timothy Baker', 'Jemima Farrell'), ('Tim Boothe', 'Tim Rownam'), ('Gerald Butters', 'Darren Smith'), ('Joan Coplin', 'Timothy Baker'), ('Erica Crumpet', 'Tracy Smith'), ('Nancy Dare', 'Janice Joplette'), ('Matthew Genting', 'Gerald Butters'), ('John Hunt', 'Millicent Purview'), ('David Jones', 'Janice Joplette'), ('Douglas Jones', 'David Jones'), ('Janice Joplette', 'Darren Smith'), ('Anna Mackenzie', 'Darren Smith'), ('Charles Owen', 'Darren Smith'), ('David Pinker', 'Jemima Farrell'), ('Millicent Purview', 'Tracy Smith'), ('Henrietta Rumney', 'Matthew Genting'), ('Ramnaresh Sarwin', 'Florence Bader'), ('Jack Smith', 'Darren Smith'), ('Ponder Stibbons', 'Burton Tracy'), ('Henry Worthington-Smyth', 'Tracy Smith')]


In [32]:
query1 = """
select f.name, count(b.memid) as Useage
from bookings as b
left join facilities as f
on f.facid = b.facid
where b.memid <> 0
group by f.name
order by f.name
"""
rs = con.execute(query1)
print(rs.fetchall())

[('Badminton Court', 344), ('Massage Room 1', 421), ('Massage Room 2', 27), ('Pool Table', 783), ('Snooker Table', 421), ('Squash Court', 195), ('Table Tennis', 385), ('Tennis Court 1', 308), ('Tennis Court 2', 276)]


In [38]:
query1 = """
select substr(b.starttime,1, 7) as `Year-Month`, count(b.memid) as Useage 
from bookings as b
left join facilities as f
on f.facid = b.facid
where b.memid <> 0
group by `Year-Month`
order by `Year-Month`
"""
rs = con.execute(query1)
print(rs.fetchall())

[('2012-07', 480), ('2012-08', 1168), ('2012-09', 1512)]
